In [1]:
from census import Census
from us import states
from dotenv import load_dotenv
from bson import ObjectId
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent='geospatial-data-project')
import pgeocode
import os
import pickle

In [2]:
from pymongo import MongoClient
client = MongoClient()
db = client.get_database('companies_project')

# Feeding companies_usa with census data

In [137]:
census_api_k = os.getenv('census_api_key')
c = Census(census_api_k, year=2018)

In [138]:
median_age = 'DP05_0018E'
range_25_34 = 'DP05_0010PE'
range_35_44 = 'DP05_0011PE'

school_enroll = 'DP02_0052E'
school_enrollP = 'DP02_0052PE'

preschool_enroll = 'DP02_0053E'
preschool_enrollP = 'DP02_0053PE'

kinderg_enroll = 'DP02_0054E'
kinderg_enrollP = 'DP02_0054PE'

elem_enroll = 'DP02_0055E'
elem_enrollP = 'DP02_0055PE'

high_enroll = 'DP02_0056E'
high_enrollP = 'DP02_0056PE'

hschool_gradP = 'DP02_0066PE'
bach_gradP = 'DP02_0067PE'

In [255]:
zip_code = '94588'
#c.acs5dp.state_county(median_age, 'Fresno County')

## Get zips from USA

In [271]:
empty_to_none = db.companies_usa.update_many({'offices.zip_code':''}, {'$set':{'offices.zip_code':None}})

In [272]:
empty_to_none.raw_result

{'n': 637, 'nModified': 637, 'ok': 1.0, 'updatedExisting': True}

In [317]:
zips = list(db.companies_usa.find({'offices.zip_code':None}, {'_id':1, 'offices.latitude':1, 'offices.longitude':1}))

In [290]:
leftovers_zips_usa = []
no_post_code = []

In [297]:
for zip in zips:
    try:
        latitude = zip['offices']['latitude']
        longitude = zip['offices']['longitude']
        id = ObjectId(zip['_id'])
        location = geolocator.reverse(f"{latitude},{longitude}")
        db.companies_usa.update_one({'_id':id}, {'$set':{'offices.zip_code': location.raw['address']['postcode']}})
        leftovers_zips_usa.append({'_id':id, 'loc_params':location.raw})
    except:
        no_post_code.append({'_id':id, 'coord':{'latitude':latitude, 'longitude':longitude}})

In [315]:
cwd = os.getcwd()
with open(cwd+'/data/usa_docs/leftovers_zips_usa.txt', 'wb') as leftov:
    pickle.dump(leftovers_zips_usa, leftov)
with open(cwd+'/data/usa_docs/no_post_codes.txt', 'wb') as no_post:
    pickle.dump(no_post_code, no_post)

## Get zips from Europe